# Import Dependencies

In [1]:
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
import time

# Init Splinter Browser

In [3]:
# MAC
# executable_path = { 'executable_path': '/usr/local/bin/chromedriver' }
# WINDOWS
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

# Scrape the List of Genre 

In [9]:
# URL to be scraped
url = 'https://www.vgchartz.com/gamedb/?page='

# Open webpage
browser.visit(url)

# Retrieve HTML webpage source
html = browser.html

# Parse HTML webpage source using BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Scrape the list of genre
genre_list = []
result_select = soup.find('select', {'name':'genre'})
result_options = result_select.find_all('option')
for result in result_options:
    if result['value'] != '':
        genre_list.append(result['value'])
genre_list

['Action',
 'Action-Adventure',
 'Adventure',
 'Board Game',
 'Education',
 'Fighting',
 'Misc',
 'MMO',
 'Music',
 'Party',
 'Platform',
 'Puzzle',
 'Racing',
 'Role-Playing',
 'Sandbox',
 'Shooter',
 'Simulation',
 'Sports',
 'Strategy',
 'Visual Novel']

# Scrape Games data for each Genre